In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%tensorflow_version 1.x
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm
from glob import glob
import time, gc
import cv2
from tensorflow import keras
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import clone_model
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import seaborn as sns
import albumentations as A

In [0]:
TRAIN = ['drive/My Drive/Neural NotWork/Data/train_image_data_0.parquet',
         'drive/My Drive/Neural NotWork/Data/train_image_data_1.parquet',
         'drive/My Drive/Neural NotWork/Data/train_image_data_2.parquet',
         'drive/My Drive/Neural NotWork/Data/train_image_data_3.parquet']

In [0]:
TEST =  ['drive/My Drive/Neural NotWork/Data/test_image_data_0.parquet',
         'drive/My Drive/Neural NotWork/Data/test_image_data_1.parquet',
         'drive/My Drive/Neural NotWork/Data/test_image_data_2.parquet',
         'drive/My Drive/Neural NotWork/Data/test_image_data_3.parquet']

In [0]:
train_df_ = pd.read_csv('drive/My Drive/Neural NotWork/Data/train.csv')
sample_sub_df = pd.read_csv('drive/My Drive/Neural NotWork/Data/sample_submission.csv')

# not really useful, just for human understanding
class_map_df = pd.read_csv('drive/My Drive/Neural NotWork/Data/class_map.csv')

In [0]:
# train_X0 = pd.read_parquet(TRAIN[0],'auto')
# train_X1 = pd.read_parquet(TRAIN[1],'auto')
# train_X2 = pd.read_parquet(TRAIN[2],'auto')
# train_X3 = pd.read_parquet(TRAIN[3],'auto')

In [0]:
HEIGHT = 137
WIDTH = 236

In [0]:
def resize(df, size=64, need_progress_bar=True):
    resized = {}
    resize_size=112
    angle=0
    if need_progress_bar:
        for i in tqdm(range(df.shape[0])):
            #image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size),None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
            image=df.loc[df.index[i]].values.reshape(137,236)
            #Centering
            image_center = tuple(np.array(image.shape[1::-1]) / 2)
            matrix = cv2.getRotationMatrix2D(image_center, angle, 1.0)
            image = cv2.warpAffine(image, matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR,
                            borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
            #Scaling
            matrix = cv2.getRotationMatrix2D(image_center, 0, 1.0)
            image = cv2.warpAffine(image, matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR,
                            borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
            # #Removing Blur
            # aug = A.GaussianBlur(p=1.0)
            # image = aug(image=image)['image']
            # #Noise Removing
            # augNoise=A.MultiplicativeNoise(p=1.0)
            # image = augNoise(image=image)['image']
            # #Removing Distortion
            # augDist=A.ElasticTransform(sigma=50, alpha=1, alpha_affine=10, p=1.0)
            # image = augDist(image=image)['image']
            #Brightness
            augBright=A.RandomBrightnessContrast(p=1.0)
            image = augBright(image=image)['image']
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            #image=affine_image(image)
            #image= crop_resize(image)
            #image = cv2.resize(image,(size,size),interpolation=cv2.INTER_AREA)
            #image=resize_image(image,(64,64))
            #image = cv2.resize(image,(size,size),interpolation=cv2.INTER_AREA)
            #gaussian_3 = cv2.GaussianBlur(image, (5,5), cv2.BORDER_DEFAULT) #unblur
            #image = cv2.addWeighted(image, 1.5, gaussian_3, -0.5, 0, image)
            #kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
            #image = cv2.filter2D(image, -1, kernel)
            #ret,image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
            resized[df.index[i]] = resized_roi.reshape(-1)
    else:
        for i in range(df.shape[0]):
            #image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size),None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
            image=df.loc[df.index[i]].values.reshape(137,236)
            image_center = tuple(np.array(image.shape[1::-1]) / 2)
            matrix = cv2.getRotationMatrix2D(image_center, angle, 1.0)
            image = cv2.warpAffine(image, matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR,
                            borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
            matrix = cv2.getRotationMatrix2D(image_center, 0, 1.0)
            image = cv2.warpAffine(image, matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR,
                            borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
            #Removing Blur
            #aug = A.GaussianBlur(p=1.0)
            #image = aug(image=image)['image']
            #Noise Removing
            #augNoise=A.MultiplicativeNoise(p=1.0)
            #image = augNoise(image=image)['image']
            #Removing Distortion
            #augDist=A.ElasticTransform(sigma=50, alpha=1, alpha_affine=10, p=1.0)
            #image = augDist(image=image)['image']
            #Brightness
            augBright=A.RandomBrightnessContrast(p=1.0)
            image = augBright(image=image)['image']
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            #image=affine_image(image)
            #image= crop_resize(image)
            #image = cv2.resize(image,(size,size),interpolation=cv2.INTER_AREA)
            #image=resize_image(image,(64,64))
            #image = cv2.resize(image,(size,size),interpolation=cv2.INTER_AREA)
            #gaussian_3 = cv2.GaussianBlur(image, (5,5), cv2.BORDER_DEFAULT) #unblur
            #image = cv2.addWeighted(image, 1.5, gaussian_3, -0.5, 0, image)
            #kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
            #image = cv2.filter2D(image, -1, kernel)
            #ret,image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
            resized[df.index[i]] = resized_roi.reshape(-1)
    resized = pd.DataFrame(resized).T
    return resized

In [0]:
def get_dummies(df):
    cols = []
    for col in df:
        cols.append(pd.get_dummies(df[col].astype(str)))
    return pd.concat(cols, axis=1)

In [0]:
IMG_SIZE=112
N_CHANNELS=1

In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D

inputs = Input(shape = (IMG_SIZE, IMG_SIZE, 1))

# Begin
model = Conv2D(filters=256, kernel_size=(4, 4), padding='SAME', strides= 2, activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1))(inputs)
model = MaxPool2D(pool_size=(3, 3), strides= 2)(model)

# Dense Block 1
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=64, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=64, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=64, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=64, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=64, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=64, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)

# Transition layers 1
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=64, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = AveragePooling2D(pool_size=(2, 2), strides= 2)(model)

# Dense Block 2
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu')(model)


# Transition layers 2
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=128, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = AveragePooling2D(pool_size=(2, 2), strides= 2)(model)


# Dense Block 3
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu')(model)


# Transition layers 3
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=256, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = AveragePooling2D(pool_size=(2, 2), strides= 2)(model)


# Dense Block 4
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(1, 1), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME', activation='relu')(model)


# Final Pool
model = GlobalAveragePooling2D()(model)

head_root = Dense(168, activation = 'softmax', name = 'roots')(model)
head_vowel = Dense(11, activation = 'softmax', name = 'vowels')(model)
head_consonant = Dense(7, activation = 'softmax', name = 'consonants')(model)

model = Model(inputs=inputs, outputs=[head_root, head_vowel, head_consonant])

In [0]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 112, 112, 1)  0                                            
__________________________________________________________________________________________________
conv2d_113 (Conv2D)             (None, 56, 56, 256)  4352        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 27, 27, 256)  0           conv2d_113[0][0]                 
__________________________________________________________________________________________________
batch_normalization_112 (BatchN (None, 27, 27, 256)  1024        max_pooling2d_2[0][0]            
____________________________________________________________________________________________

In [0]:
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
learning_rate_reduction_root = ReduceLROnPlateau(monitor='roots_accuracy', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)
learning_rate_reduction_vowel = ReduceLROnPlateau(monitor='vowels_accuracy', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)
learning_rate_reduction_consonant = ReduceLROnPlateau(monitor='consonants_accuracy', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)

In [0]:
#decreased batch size
batch_size = 256
epochs = 6

In [0]:
class MultiOutputDataGenerator(keras.preprocessing.image.ImageDataGenerator):

    def flow(self,
             x,
             y=None,
             batch_size=32,
             shuffle=True,
             sample_weight=None,
             seed=None,
             save_to_dir=None,
             save_prefix='',
             save_format='png',
             subset=None):

        targets = None
        target_lengths = {}
        ordered_outputs = []
        for output, target in y.items():
            if targets is None:
                targets = target
            else:
                targets = np.concatenate((targets, target), axis=1)
            target_lengths[output] = target.shape[1]
            ordered_outputs.append(output)


        for flowx, flowy in super().flow(x, targets, batch_size=batch_size,
                                         shuffle=shuffle):
            target_dict = {}
            i = 0
            for output in ordered_outputs:
                target_length = target_lengths[output]
                target_dict[output] = flowy[:, i: i + target_length]
                i += target_length

            yield flowx, target_dict

In [0]:
## run this before retrain modified model 
keras.backend.backend()

'tensorflow'

In [0]:
histories = []
for i in range(1):
    train_df = pd.merge(pd.read_parquet(f'drive/My Drive/Neural NotWork/Data/train_image_data_{i}.parquet'), train_df_, on='image_id').drop(['image_id'], axis=1)
    # Visualize few samples of current training dataset
    # fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(16, 8))
    # count=0
    # for row in ax:
    #     for col in row:
    #         col.imshow(resize(train_df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'], axis=1).iloc[[count]], need_progress_bar=False).values.reshape(IMG_SIZE, IMG_SIZE))
    #         count += 1
    # plt.show()
    
    X_train = train_df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic', 'grapheme'], axis=1)
    X_train = resize(X_train)
    
    # CNN takes images in shape `(batch_size, h, w, channels)`, so reshape the images
    X_train = X_train.values.reshape(-1, IMG_SIZE, IMG_SIZE, N_CHANNELS)
    
    Y_train_root = pd.get_dummies(train_df['grapheme_root']).values
    Y_train_vowel = pd.get_dummies(train_df['vowel_diacritic']).values
    Y_train_consonant = pd.get_dummies(train_df['consonant_diacritic']).values

    print(f'Training images: {X_train.shape}')
    print(f'Training labels root: {Y_train_root.shape}')
    print(f'Training labels vowel: {Y_train_vowel.shape}')
    print(f'Training labels consonants: {Y_train_consonant.shape}')

    # Divide the data into training and validation set
    x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant = train_test_split(X_train, Y_train_root, Y_train_vowel, Y_train_consonant, test_size=0.08, random_state=124)
    del train_df
    del X_train
    del Y_train_root, Y_train_vowel, Y_train_consonant

    # Data augmentation for creating more training data
    datagen = MultiOutputDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=8,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.15, # Randomly zoom image 
        width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


    # This will just calculate parameters required to augment the given data. This won't perform any augmentations
    datagen.fit(x_train)

    # Fit the model
    history = model.fit_generator(datagen.flow(x_train, {'roots': y_train_root, 'vowels': y_train_vowel, 'consonants': y_train_consonant}, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test, [y_test_root, y_test_vowel, y_test_consonant]), 
                              steps_per_epoch=x_train.shape[0] // batch_size, 
                              callbacks=[learning_rate_reduction_root, learning_rate_reduction_vowel, learning_rate_reduction_consonant])

    histories.append(history)
    
    # Delete to reduce memory usage
    del x_train
    del x_test
    del y_train_root
    del y_test_root
    del y_train_vowel
    del y_test_vowel
    del y_train_consonant
    del y_test_consonant
    gc.collect()


Training images: (50210, 112, 112, 1)
Training labels root: (50210, 168)
Training labels vowel: (50210, 11)
Training labels consonants: (50210, 7)
Epoch 1/6
180/180 [==============================] - 128s 712ms/step - loss: 8.3319 - roots_loss: 4.7769 - vowels_loss: 2.2390 - consonants_loss: 1.3160 - roots_acc: 0.0270 - vowels_acc: 0.1984 - consonants_acc: 0.6022 - val_loss: 9.8551 - val_roots_loss: 5.4341 - val_vowels_loss: 2.9074 - val_consonants_loss: 1.5136 - val_roots_acc: 0.0301 - val_vowels_acc: 0.1992 - val_consonants_acc: 0.6221


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1379: RuntimeWarning: Reduce LR on plateau conditioned on metric `roots_accuracy` which is not available. Available metrics are: val_loss,val_roots_loss,val_vowels_loss,val_consonants_loss,val_roots_acc,val_vowels_acc,val_consonants_acc,loss,roots_loss,vowels_loss,consonants_loss,roots_acc,vowels_acc,consonants_acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1379: RuntimeWarning: Reduce LR on plateau conditioned on metric `vowels_accuracy` which is not available. Available metrics are: val_loss,val_roots_loss,val_vowels_loss,val_consonants_loss,val_roots_acc,val_vowels_acc,val_consonants_acc,loss,roots_loss,vowels_loss,consonants_loss,roots_acc,vowels_acc,consonants_acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1379: RuntimeWarning: Reduce LR on plateau conditioned on met

Epoch 2/6
180/180 [==============================] - 89s 493ms/step - loss: 8.4767 - roots_loss: 4.8785 - vowels_loss: 2.2932 - consonants_loss: 1.3050 - roots_acc: 0.0267 - vowels_acc: 0.2037 - consonants_acc: 0.6213 - val_loss: 9.1001 - val_roots_loss: 5.1271 - val_vowels_loss: 2.6004 - val_consonants_loss: 1.3726 - val_roots_acc: 0.0286 - val_vowels_acc: 0.1992 - val_consonants_acc: 0.6221
Epoch 3/6
180/180 [==============================] - 88s 490ms/step - loss: 8.1301 - roots_loss: 4.7484 - vowels_loss: 2.1697 - consonants_loss: 1.2120 - roots_acc: 0.0270 - vowels_acc: 0.2091 - consonants_acc: 0.6255 - val_loss: 8.5244 - val_roots_loss: 4.8915 - val_vowels_loss: 2.3519 - val_consonants_loss: 1.2810 - val_roots_acc: 0.0256 - val_vowels_acc: 0.2014 - val_consonants_acc: 0.6221
Epoch 4/6
180/180 [==============================] - 88s 491ms/step - loss: 8.1356 - roots_loss: 4.7467 - vowels_loss: 2.1735 - consonants_loss: 1.2154 - roots_acc: 0.0269 - vowels_acc: 0.2070 - consonants_ac

In [0]:
import os
model_name = 'DenseNet_0303.h5'
save_dir = os.path.join("drive/My Drive/Neural NotWork/", 'saved_models')
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at drive/My Drive/Neural NotWork/saved_models/DenseNet_0303.h5 
